In [1]:
%run "get_packages.py"

C:\Program64\Python\Python\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
today = date.today()
end_date = today.strftime("%Y%m%d")
strt_date="20241030"
std_size = 0.03
concentration_size = 0.05
step_size = 0.005
TE_upper_bound = 0.012
TE_lower_bound = 0.008
features_weight = 0.33
multiplier = 2
carry_cost_lim = 0.003
start_date = 1/1/2003

REER_lag = 22
TB_lag = 30
GDP_lag = 120
CPI_lag = 30

Adj_REER = ["REER", "Conditional REER", "Symmetrical REER"]
Carry = ["Nominal Carry", "Real Carry"]
Conviction = ["Awa", "Agn"]

currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]


In [3]:
#SPOT PRICE

bquery = blp.BlpQuery().start()
spot_query = bquery.bdh(
        ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy", "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy", "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy", "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy", "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy", "KRW CMPN Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

spot_query['security_short'] = spot_query['security'].str[:3]

spot = spot_query.pivot_table(index='date', columns='security_short', values='PX_LAST', aggfunc='first')
spot.reset_index( inplace=True)
spot.rename(columns={'date':'DATE'}, inplace=True)
spot['DATE'] = pd.to_datetime(spot['DATE'])
spot.sort_values(by='DATE', ascending=False, inplace=True)
spot.dropna(inplace=True)

# Extract relevant columns
columns = ["DATE"] + currencies
spot_subset = spot[columns]

# Calculate daily returns
spot_subset.set_index("DATE", inplace=True)
daily_return = spot_subset.shift(-1)/spot_subset  -1
daily_return.reset_index(inplace=True)
#daily_return.dropna(inplace=True)
spot_return = daily_return.melt(id_vars=["DATE"], var_name="Currency", value_name="Daily_Return")
spot_return

,DATE,Currency,Daily_Return
0,2024-11-19,BRL,-0.003844
1,2024-11-18,BRL,0.007649
2,2024-11-15,BRL,0.000000
3,2024-11-14,BRL,0.000173
4,2024-11-13,BRL,-0.006968
...,...,...,...
310,2024-11-05,KRW,-0.004082
311,2024-11-04,KRW,0.005075
312,2024-11-01,KRW,-0.004824
313,2024-10-31,KRW,0.004149


In [4]:
bquery = blp.BlpQuery().start()
MA_query = bquery.bdh(
        ["BRL Curncy","CLP Curncy","CNY Curncy","COP Curncy","CZK Curncy","HUF Curncy","IDR Curncy","MYR Curncy","MXN Curncy","PEN Curncy","PHP Curncy","PLN Curncy","RON Curncy","RUB Curncy","ZAR Curncy","THB Curncy","TRY Curncy","INR Curncy","SGD Curncy","TWD Curncy","KRW Curncy"],
        ["MOV_AVG_5D","MOV_AVG_20D"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})
MA_query.rename(columns={'date':'DATE'}, inplace=True)
MA_query['Currency'] = MA_query['security'].str[:3]
MA_query.drop(columns="security", inplace=True)
MA_query['Momentum'] = np.where(MA_query['MOV_AVG_5D'] > MA_query['MOV_AVG_20D'], 1, -1)
Momentum = MA_query[["DATE", "Currency", "MOV_AVG_5D", "MOV_AVG_20D","Momentum"]]
Momentum['DATE'] = pd.to_datetime(Momentum['DATE'])
Momentum.sort_values(by='DATE', ascending=False, inplace=True)
Momentum.reset_index(drop=True, inplace=True)
#Momentum.dropna(inplace=True)
Momentum

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)


,DATE,Currency,MOV_AVG_5D,MOV_AVG_20D,Momentum
0,2024-11-14,MYR,4.4328,4.3742,1
1,2024-11-14,BRL,5.7686,5.7386,1
2,2024-11-14,INR,84.3890,84.1838,1
3,2024-11-14,SGD,1.3373,1.3247,1
4,2024-11-14,THB,34.5920,33.9230,1
...,...,...,...,...,...
247,2024-10-30,IDR,15682.0000,15590.0000,1
248,2024-10-30,SGD,1.3217,1.3124,1
249,2024-10-30,MYR,4.3619,4.3096,1
250,2024-10-30,PEN,3.7643,3.7551,1


In [6]:
#3M Implied vol
bquery = blp.BlpQuery().start()
IMPVOL_query = bquery.bdh(
        ["USDBRLV3M Curncy","USDCLPV3M Curncy","USDCNYV3M Curncy","USDCOPV3M Curncy","USDCZKV3M Curncy","USDHUFV3M Curncy","USDIDRV3M Curncy","USDMYRV3M Curncy","USDMXNV3M Curncy","USDPENV3M Curncy","USDPHPV3M Curncy","USDPLNV3M Curncy","USDRONV3M Curncy","USDRUBV3M Curncy","USDZARV3M Curncy","USDTHBV3M Curncy","USDTRYV3M Curncy","USDINRV3M Curncy","USDSGDV3M Curncy","USDTWDV3M Curncy","USDKRWV3M Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"}) #

IMPVOL_query.rename(columns={'date':'DATE', 'PX_LAST':'3M Implied Vol'}, inplace=True)

conditions = [
    (IMPVOL_query['security'] == 'USDBRLV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCLPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCNYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCOPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCZKV3M Curncy'),
    (IMPVOL_query['security'] == 'USDHUFV3M Curncy'),
    (IMPVOL_query['security'] == 'USDIDRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMYRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMXNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPENV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPHPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPLNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRONV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRUBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDZARV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTHBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTRYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDINRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDSGDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTWDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDKRWV3M Curncy'),
]

IMPVOL_query['Currency'] = np.select(conditions, currencies, default=None)
IMPVOL_query.drop(columns="security", inplace=True)
IMPVOL_query.sort_values(by='DATE', ascending=False, inplace=True)
IMPVOL_query=IMPVOL_query[['DATE','Currency','3M Implied Vol']]
IMPVOL_query

,DATE,Currency,3M Implied Vol
251,2024-11-14,KRW,9.1250
241,2024-11-14,HUF,11.5400
231,2024-11-14,CLP,13.4300
232,2024-11-14,CNY,6.0350
233,2024-11-14,THB,9.5750
...,...,...,...
17,2024-10-30,TWD,8.6000
18,2024-10-30,CNY,7.2950
19,2024-10-30,KRW,11.0800
20,2024-10-30,PEN,5.5125


In [8]:
import pandas as pd
from pandas.tseries.offsets import BDay
import numpy as np

all_currencies = ["BRL","CLP","CNY","COP","CZK","HUF","IDR","MYR","MXN","PEN","PHP","PLN","RON","RUB","ZAR","THB","TRY","INR","SGD","TWD","KRW","UAH","EGP","EUR","JPY","USD","GBP","CAD","ARS","CHF"]

# Create an empty list to store the dataframes
all_cpi_data = []

# Loop through the dates from 2020 to today
start_date = pd.Timestamp('2022-01-01')
end_date = pd.Timestamp('today') 
for date in pd.date_range(start=start_date, end=end_date, freq='MS'): # 'MS' for month start frequency
    specific_date = date + pd.DateOffset(months=1) - pd.DateOffset(days=1)  # Get the last day of the month

    begin = ((specific_date - pd.DateOffset(months=2)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')
    end = ((specific_date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1) - BDay(1)).strftime('%Y%m%d')

    bquery = blp.BlpQuery().start()
    CPI_query = bquery.bdh(
            ["BZPIIPCM Index","CLINNSMO Index","CNCPIMOM Index","COCPIMOM Index","CZCPMOM Index","HUCPIMM Index","IDCPIM Index","MACPIMOM Index","MXCPCHNG Index","PRCPNONC Index","PHC2MOM Index","POCPIMOM Index","ROCOPMOM Index","RUCPIMOM Index","SACPIMOM Index","THCPIMOM Index","TUCPIM Index","INFUTOT Index","SICPIMOM Index","TWCPIMOM Index","KOCPIMOM Index","UACPTMOM Index","EGCPMOM Index","ECCPEMUM Index","JNCPIMOM Index","CPURNSA% Index","UKRPCJMR Index","CACPICHG Index","ARNCNINX Index","SZCPIMOM Index"],
            ["PX_LAST"],
            start_date= begin,
            end_date= end, 
            options={"adjustmentSplit": True, "nonTradingDayFillOption":"ALL_CALENDAR_DAYS", "periodicitySelection":"MONTHLY", 'nonTradingDayFillMethod':'NIL_VALUE'})
    CPI_query.rename(columns={'PX_LAST':'CPI'}, inplace=True)

    conditions = [
    (CPI_query['security'] == 'BZPIIPCM Index'),
    (CPI_query['security'] == 'CLINNSMO Index'),
    (CPI_query['security'] == 'CNCPIMOM Index'),
    (CPI_query['security'] == 'COCPIMOM Index'),
    (CPI_query['security'] == 'CZCPMOM Index'),
    (CPI_query['security'] == 'HUCPIMM Index'),
    (CPI_query['security'] == 'IDCPIM Index'),
    (CPI_query['security'] == 'MACPIMOM Index'),
    (CPI_query['security'] == 'MXCPCHNG Index'),
    (CPI_query['security'] == 'PRCPNONC Index'),
    (CPI_query['security'] == 'PHC2MOM Index'),
    (CPI_query['security'] == 'POCPIMOM Index'),
    (CPI_query['security'] == 'ROCOPMOM Index'),
    (CPI_query['security'] == 'RUCPIMOM Index'),
    (CPI_query['security'] == 'SACPIMOM Index'),
    (CPI_query['security'] == 'THCPIMOM Index'),
    (CPI_query['security'] == 'TUCPIM Index'),
    (CPI_query['security'] == 'INFUTOT Index'),
    (CPI_query['security'] == 'SICPIMOM Index'),
    (CPI_query['security'] == 'TWCPIMOM Index'),
    (CPI_query['security'] == 'KOCPIMOM Index'),
    (CPI_query['security'] == 'UACPTMOM Index'),
    (CPI_query['security'] == 'EGCPMOM Index'),
    (CPI_query['security'] == 'ECCPEMUM Index'),
    (CPI_query['security'] == 'JNCPIMOM Index'),
    (CPI_query['security'] == 'CPURNSA% Index'),
    (CPI_query['security'] == 'UKRPCJMR Index'),
    (CPI_query['security'] == 'CACPICHG Index'),
    (CPI_query['security'] == 'ARNCNINX Index'),
    (CPI_query['security'] == 'SZCPIMOM Index')
    ]

    CPI_query['Currency'] = np.select(conditions, all_currencies, default=None)
    CPI_query.loc[CPI_query['Currency'] == 'INR', 'CPI'] = 100*(CPI_query['CPI']/185.8 -1)
        
    CPI_query=CPI_query[['Currency','CPI']]
    CPI_query['Date'] = specific_date # Add a Date column

    all_cpi_data.append(CPI_query) # Append the dataframe to the list


# Concatenate all dataframes vertically
final_cpi_data = pd.concat(all_cpi_data, ignore_index=True)

print(final_cpi_data)

     Currency      CPI       Date
0         TRY  3.51000 2022-01-31
1         HUF  0.70000 2022-01-31
2         CZK  0.20000 2022-01-31
3         COP  0.50000 2022-01-31
4         CNY  0.40000 2022-01-31
...       ...      ...        ...
1045      TWD  0.11000 2024-11-30
1046      SGD  0.30000 2024-11-30
1047      INR  4.52099 2024-11-30
1048      ARS  3.50000 2024-11-30
1049      CHF -0.30000 2024-11-30

[1050 rows x 3 columns]


In [26]:
from pandas.tseries.offsets import MonthEnd

# Calculate the start and end dates for the entire period (month-end frequency)
start_date = final_cpi_data['Date'].min()
end_date = final_cpi_data['Date'].max()

# Create a daily date range for the entire period
daily_dates = pd.date_range(start=start_date - pd.DateOffset(days=1), end=end_date, freq='D')

# Create an empty dataframe for the daily CPI data
final_daily_cpi_data = pd.DataFrame(columns=['Date', 'Currency', 'CPI'])

# Loop through each month-end date in the CPI_query
for month_end_date in final_cpi_data['Date'].unique():
    # Filter the CPI_query for the current month-end
    month_data = final_cpi_data[final_cpi_data['Date'] == month_end_date]

    # Get the start and end dates for the current month
    start_of_month = month_end_date.replace(day=1)

    # Filter the daily_dates for the current month
    current_month_dates = daily_dates[(daily_dates >= start_of_month) & (daily_dates <= month_end_date)]

    # Repeat the month's CPI data for each day in the month (using cross-join)
    repeated_data = pd.merge(pd.DataFrame({'Date': current_month_dates}), month_data, how='cross')

    # Append the repeated data to the final dataframe
    final_daily_cpi_data = pd.concat([final_daily_cpi_data, repeated_data], ignore_index=True)

# Sort the final dataframe by date
final_daily_cpi_data = final_daily_cpi_data.sort_values('Date').reset_index(drop=True)

print(final_daily_cpi_data)


C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_32256/2254841379.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_daily_cpi_data = pd.concat([final_daily_cpi_data, repeated_data], ignore_index=True)


      Date Currency       CPI     Date_x     Date_y
0      NaN      TRY  3.510000 2022-01-30 2022-01-31
1      NaN      RUB  0.960000 2022-01-30 2022-01-31
2      NaN      RON  0.000000 2022-01-30 2022-01-31
3      NaN      PLN  1.000000 2022-01-30 2022-01-31
4      NaN      PHP  0.600000 2022-01-30 2022-01-31
...    ...      ...       ...        ...        ...
30175  NaN      SGD  0.700000 2024-10-31 2024-10-31
30176  NaN      INR  3.875135 2024-10-31 2024-10-31
30177  NaN      TRY  2.470000 2024-10-31 2024-10-31
30178  NaN      IDR -0.030000 2024-10-31 2024-10-31
30179  NaN      MYR  0.100000 2024-10-31 2024-10-31

[30180 rows x 5 columns]


In [5]:
start_date = pd.Timestamp('2023-11-20') 
end_date = pd.Timestamp('2024-01-01')

# Create an empty list to store the daily dataframes
all_reer_data = []

# Loop through the dates from 2022 to today (using daily frequency)
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    bquery = blp.BlpQuery().start()

    REER_query = bquery.bdp(
            ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"],
            ["PX_LAST", "LAST_UPDATE_DT"]  # Retrieve LAST_UPDATE_DT directly
            )
    REER_query.rename(columns={'PX_LAST':'REER'}, inplace=True)

    conditions = [

    (REER_query['security'] == 'BISBBRR Index'),
    (REER_query['security'] == 'BISBCLR Index'),
    (REER_query['security'] == 'BISBCNR Index'),
    (REER_query['security'] == 'BISBCOP Index'),
    (REER_query['security'] == 'BISBCZR Index'),
    (REER_query['security'] == 'BISBHUR Index'),
    (REER_query['security'] == 'BISBIDR Index'),
    (REER_query['security'] == 'BISBMYR Index'),
    (REER_query['security'] == 'BISBMXR Index'),
    (REER_query['security'] == 'BISBPER Index'),
    (REER_query['security'] == 'BISBPHR Index'),
    (REER_query['security'] == 'BISBPLR Index'),
    (REER_query['security'] == 'BISBROR Index'),
    (REER_query['security'] == 'BISBRUR Index'),
    (REER_query['security'] == 'BISBZAR Index'),
    (REER_query['security'] == 'BISBTHR Index'),
    (REER_query['security'] == 'BISBTRR Index'),
    (REER_query['security'] == 'BISBINR Index'),
    (REER_query['security'] == 'BISBSGR Index'),
    (REER_query['security'] == 'BISBTWR Index'),
    (REER_query['security'] == 'BISBKRR Index')    ]

    REER_query['Currency'] = np.select(conditions, currencies, default=None) 

    def get_5Y_average(row): # Modified function to work on rows
        security = row['security']
        LAST_UPDATE = row['LAST_UPDATE_DT']
        start_date = (LAST_UPDATE - relativedelta(months=60)).strftime('%Y-%m-%d')
        end_date = LAST_UPDATE.strftime('%Y-%m-%d')
        query_text = f"get(avg(px_last(dates=range({start_date}, {end_date}), Frq=M, fill=PREV))) for(['{security}'])"
        BQL_query = bquery.bql(expression=f"{query_text}")
        try:
            avg_value = BQL_query.loc[BQL_query['security'] == security, 'value'].iloc[0]
        except IndexError:
            avg_value = None
        return avg_value

    REER_query['REER_5Y_avg'] = REER_query.apply(get_5Y_average, axis=1)
    REER_query.reset_index(inplace=True)
    REER_query=REER_query[['Currency', 'REER', 'REER_5Y_avg']]
    REER_query['Date'] = date  # Add the date column

    all_reer_data.append(REER_query)

# Concatenate all daily dataframes vertically
final_reer_data = pd.concat(all_reer_data, ignore_index=True)

print(final_reer_data)

    Currency    REER REER_5Y_avg       Date
0        KRW   94.29        None 2023-11-20
1        BRL  103.74        None 2023-11-20
2        CLP  101.65        None 2023-11-20
3        CNY   91.29        None 2023-11-20
4        COP  109.20        None 2023-11-20
..       ...     ...         ...        ...
898      THB   94.44        None 2024-01-01
899      TRY   94.94        None 2024-01-01
900      INR  101.43        None 2024-01-01
901      SGD  116.18        None 2024-01-01
902      TWD   96.75        None 2024-01-01

[903 rows x 4 columns]


In [6]:
final_reer_data

NameError: name 'final_reer_data' is not defined

In [93]:
LAST_UPDATE

Timestamp('2024-08-31 00:00:00')

In [112]:
#REER discount 
bquery = blp.BlpQuery().start()

REER_query = bquery.bdp(
        ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"],
        ["PX_LAST"])
REER_query.rename(columns={'PX_LAST':'REER'}, inplace=True)

conditions = [
    (REER_query['security'] == 'BISBBRR Index'),
    (REER_query['security'] == 'BISBCLR Index'),
    (REER_query['security'] == 'BISBCNR Index'),
    (REER_query['security'] == 'BISBCOP Index'),
    (REER_query['security'] == 'BISBCZR Index'),
    (REER_query['security'] == 'BISBHUR Index'),
    (REER_query['security'] == 'BISBIDR Index'),
    (REER_query['security'] == 'BISBMYR Index'),
    (REER_query['security'] == 'BISBMXR Index'),
    (REER_query['security'] == 'BISBPER Index'),
    (REER_query['security'] == 'BISBPHR Index'),
    (REER_query['security'] == 'BISBPLR Index'),
    (REER_query['security'] == 'BISBROR Index'),
    (REER_query['security'] == 'BISBRUR Index'),
    (REER_query['security'] == 'BISBZAR Index'),
    (REER_query['security'] == 'BISBTHR Index'),
    (REER_query['security'] == 'BISBTRR Index'),
    (REER_query['security'] == 'BISBINR Index'),
    (REER_query['security'] == 'BISBSGR Index'),
    (REER_query['security'] == 'BISBTWR Index'),
    (REER_query['security'] == 'BISBKRR Index')
]

REER_query['Currency'] = np.select(conditions, currencies, default=None)

def get_5Y_average(df):
    security = df['security']
    LAST_UPDATE = bquery.bdp( [f"{security}"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
    print(LAST_UPDATE)
    start_date = (LAST_UPDATE - relativedelta(months=62)).strftime('%Y-%m-%d')
    end_date = LAST_UPDATE.strftime('%Y-%m-%d')
    query_text = f"get(avg(px_last(dates=range({start_date}, {end_date}), Frq=M, fill=PREV))) for(['{security}'])"
    BQL_query = bquery.bql(expression=f"{query_text}")
    return BQL_query.loc[1,'value']

REER_query['REER_5Y_avg'] = REER_query.apply(get_5Y_average, axis=1)
REER_query.reset_index(inplace=True)
REER_query=REER_query[['Currency', 'REER', 'REER_5Y_avg']]

REER_query


2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00
2024-09-30 00:00:00


,Currency,REER,REER_5Y_avg
0,KRW,94.78,97.759683
1,PHP,101.08,99.865556
2,PLN,122.69,105.540317
3,RON,112.81,104.490794
4,RUB,99.49,106.499841
5,ZAR,104.61,104.122698
6,THB,97.09,96.271270
7,TRY,95.86,92.116984
8,INR,99.34,100.289683
9,SGD,116.75,105.801905


In [1]:


bquery = blp.BlpQuery().start()

currencies = ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'MYR', 'MXN', 
              'PEN', 'PHP', 'PLN', 'RON', 'RUB', 'ZAR', 'THB', 'TRY', 'INR',
              'SGD', 'TWD', 'KRW']

securities = ["BISBBRR Index","BISBCLR Index","BISBCNR Index","BISBCOP Index","BISBCZR Index","BISBHUR Index","BISBIDR Index","BISBMYR Index","BISBMXR Index","BISBPER Index","BISBPHR Index","BISBPLR Index","BISBROR Index","BISBRUR Index","BISBZAR Index","BISBTHR Index","BISBTRR Index","BISBINR Index","BISBSGR Index","BISBTWR Index","BISBKRR Index"]

def get_reer_data(date):
    """Retrieves REER data for a specific date."""
    begin = ((date - pd.DateOffset(months=62)).replace(day=1)).strftime('%Y-%m-%d') # Start_date shifted by 62months
    end = date.strftime('%Y-%m-%d')
    
    REER_query = bquery.bdh(  # Use bdh for historical data
        securities,
        ["PX_LAST"],
        start_date=begin,
        end_date=end,
        options={"periodicitySelection": "MONTHLY"} # Add options if needed
    )
    
    if REER_query.empty:
        return pd.DataFrame(columns=['Currency', 'REER', 'Date'])


    REER_query['Date'] = date
    REER_query.rename(columns={'PX_LAST':'REER'}, inplace=True)
    REER_query['Currency'] = np.select(conditions, currencies, default=None)   
    REER_query = REER_query[['Date', 'Currency', 'REER']] # Reorder columns
    return REER_query



# Set the date range
start_date = pd.Timestamp('2024-10-01')
end_date = pd.Timestamp('today')

# Create an empty list to store the dataframes
all_reer_data = []


# Get the latest available date for each security 
last_update_dates = pd.to_datetime(bquery.bdp(securities, ["LAST_UPDATE_DT"])["LAST_UPDATE_DT"]).to_dict()

for date in pd.date_range(start=start_date, end=end_date, freq='MS'):  # Month start frequency
    reer_df = get_reer_data(date)
    all_reer_data.append(reer_df)



# Concatenate all dataframes
final_reer_data = pd.concat(all_reer_data, ignore_index=True)
final_reer_data

NameError: name 'blp' is not defined

In [13]:
#Foward points (ask) : 3M and 1M
bquery = blp.BlpQuery().start()

bquery = blp.BlpQuery().start()
fpa3m_query = bquery.bdh(
        ["BCN3M BGN Curncy","CHN3M BGN Curncy","CNH3M CMPN Curncy","CLN3M BGN Curncy","CZK3M BGN Curncy","HUF3M BGN Curncy","IHN3M CMPN Curncy","MRN3M CMPN Curncy","MXN3M BGN Curncy","PSN3M BGN Curncy","PPN3M CMPN Curncy","PLN3M BGN Curncy","RON3M BGN Curncy","RUB3M BGN Curncy","ZAR3M BGN Curncy","THB3M BGN Curncy","TRY3M BGN Curncy","IRN3M CMPN Curncy","SGD3M BGN Curncy","NTN3M BGN Curncy","KWN3M CMPN Curncy"],
        ["PX_MID", "FWD_SCALE"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

fpa3m_query['Cntry_code'] = fpa3m_query['security'].str[:3]
fpa3m_query.rename(columns={'PX_MID':'FPA3M_MID'}, inplace=True)

fpa1m_query = bquery.bdp(
        ["BCN1M BGN Curncy","CHN1M BGN Curncy","CNH1M CMPN Curncy","CLN1M BGN Curncy","CZK1M BGN Curncy","HUF1M BGN Curncy","IHN1M CMPN Curncy","MRN1M CMPN Curncy","MXN1M BGN Curncy","PSN1M BGN Curncy","PPN1M CMPN Curncy","PLN1M BGN Curncy","RON1M BGN Curncy","RUB1M BGN Curncy","ZAR1M BGN Curncy","THB1M BGN Curncy","TRY1M BGN Curncy","IRN1M CMPN Curncy","SGD1M BGN Curncy","NTN1M BGN Curncy","KWN1M CMPN Curncy"],
        ["PX_MID"],
        start_date=strt_date,
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})

fpa1m_query['Cntry_code'] = fpa1m_query['security'].str[:3]
fpa1m_query.rename(columns={'PX_MID':'FPA1M_MID'}, inplace=True)

fpa_query = pd.merge(fpa3m_query, fpa1m_query, on='Cntry_code')

conditions = [
    (fpa_query['Cntry_code'] == "BCN"),
    (fpa_query['Cntry_code'] == "CHN"),
    (fpa_query['Cntry_code'] == "CNH"),
    (fpa_query['Cntry_code'] == "CLN"),
    (fpa_query['Cntry_code'] == "CZK"),
    (fpa_query['Cntry_code'] == "HUF"),
    (fpa_query['Cntry_code'] == "IHN"),
    (fpa_query['Cntry_code'] == "MRN"),
    (fpa_query['Cntry_code'] == "MXN"),
    (fpa_query['Cntry_code'] == "PSN"),
    (fpa_query['Cntry_code'] == "PPN"),
    (fpa_query['Cntry_code'] == "PLN"),
    (fpa_query['Cntry_code'] == "RON"),
    (fpa_query['Cntry_code'] == "RUB"),
    (fpa_query['Cntry_code'] == "ZAR"),
    (fpa_query['Cntry_code'] == "THB"),
    (fpa_query['Cntry_code'] == "TRY"),
    (fpa_query['Cntry_code'] == "IRN"),
    (fpa_query['Cntry_code'] == "SGD"),
    (fpa_query['Cntry_code'] == "NTN"),
    (fpa_query['Cntry_code'] == "KWN")
]

fpa_query['Currency'] = np.select(conditions, currencies, default=None)
fpa_query.sort_values(by='Currency', ascending=True, inplace=True)
fpa_query.reset_index(inplace=True)
fpa_query['FPA3M_MID']=fpa_query['FPA3M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query['FPA1M_MID']=fpa_query['FPA1M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query= fpa_query[['Currency', 'FWD_SCALE','FPA3M_MID','FPA1M_MID']]
fpa_query

TypeError: Response for TRY3M BGN Curncy contains fieldException {'fieldId': 'FWD_SCALE', 'errorInfo': {'errorInfo': {'source': '53361:rsfhdsvc3', 'code': 1, 'category': 'BAD_FLD', 'message': 'Not valid historical field', 'subcategory': 'NOT_APPLICABLE_TO_HIST_DATA'}}}

In [8]:
from xbbg import blp 
from blp import blp 
bquery = blp.BlpQuery().start() # Use xbbg.blp.BlpQuery

today = date.today()
end_date = today.strftime("%Y%m%d")
strt_date = "20221230"

fpa3m_query = bquery.bdp(
    [  # Verify these tickers!
        "BCN3M BGN Curncy", "CHN3M BGN Curncy", "CNH3M CMPN Curncy",
        "CLN3M BGN Curncy", "CZK3M BGN Curncy", "HUF3M BGN Curncy",
        "IHN3M CMPN Curncy", "MRN3M CMPN Curncy", "MXN3M BGN Curncy",
        "PSN3M BGN Curncy", "PPN3M CMPN Curncy", "PLN3M BGN Curncy",
        "RON3M BGN Curncy", "RUB3M BGN Curncy", "ZAR3M BGN Curncy",
        "THB3M BGN Curncy", "TRY3M BGN Curncy", "IRN3M CMPN Curncy",
        "SGD3M BGN Curncy", "NTN3M BGN Curncy", "KWN3M CMPN Curncy"
    ],
    ["PX_MID","FWD_SCALE"],
    options={"adjustmentSplit": True, "CDR": "5D"}  # Corrected options syntax
)

fpa3m_query 

# ... (Rest of your code to process the 'spot_query' result) ...

bquery.stop() # Important to stop the Bloomberg session when finished.


NotFoundException: Attempt to access unknown sub-element 'adjustmentSplit' on element 'ReferenceDataRequest' (0x0006000d)

In [5]:
#Foward points (ask) : 3M and 1M
bquery = blp.BlpQuery().start()
fpa3m_query = bquery.bdp(
        ["BCN3M BGN Curncy","CHN3M BGN Curncy","CNH3M CMPN Curncy","CLN3M BGN Curncy","CZK3M BGN Curncy","HUF3M BGN Curncy","IHN3M CMPN Curncy","MRN3M CMPN Curncy","MXN3M BGN Curncy","PSN3M BGN Curncy","PPN3M CMPN Curncy","PLN3M BGN Curncy","RON3M BGN Curncy","RUB3M BGN Curncy","ZAR3M BGN Curncy","THB3M BGN Curncy","TRY3M BGN Curncy","IRN3M CMPN Curncy","SGD3M BGN Curncy","NTN3M BGN Curncy","KWN3M CMPN Curncy"],
        ["PX_MID", "FWD_SCALE"])

fpa3m_query['Cntry_code'] = fpa3m_query['security'].str[:3]
fpa3m_query.rename(columns={'PX_MID':'FPA3M_MID'}, inplace=True)

fpa1m_query = bquery.bdp(
        ["BCN1M BGN Curncy","CHN1M BGN Curncy","CNH1M CMPN Curncy","CLN1M BGN Curncy","CZK1M BGN Curncy","HUF1M BGN Curncy","IHN1M CMPN Curncy","MRN1M CMPN Curncy","MXN1M BGN Curncy","PSN1M BGN Curncy","PPN1M CMPN Curncy","PLN1M BGN Curncy","RON1M BGN Curncy","RUB1M BGN Curncy","ZAR1M BGN Curncy","THB1M BGN Curncy","TRY1M BGN Curncy","IRN1M CMPN Curncy","SGD1M BGN Curncy","NTN1M BGN Curncy","KWN1M CMPN Curncy"],
        ["PX_MID"])

fpa1m_query['Cntry_code'] = fpa1m_query['security'].str[:3]
fpa1m_query.rename(columns={'PX_MID':'FPA1M_MID'}, inplace=True)

fpa_query = pd.merge(fpa3m_query, fpa1m_query, on='Cntry_code')

conditions = [
    (fpa_query['Cntry_code'] == "BCN"),
    (fpa_query['Cntry_code'] == "CHN"),
    (fpa_query['Cntry_code'] == "CNH"),
    (fpa_query['Cntry_code'] == "CLN"),
    (fpa_query['Cntry_code'] == "CZK"),
    (fpa_query['Cntry_code'] == "HUF"),
    (fpa_query['Cntry_code'] == "IHN"),
    (fpa_query['Cntry_code'] == "MRN"),
    (fpa_query['Cntry_code'] == "MXN"),
    (fpa_query['Cntry_code'] == "PSN"),
    (fpa_query['Cntry_code'] == "PPN"),
    (fpa_query['Cntry_code'] == "PLN"),
    (fpa_query['Cntry_code'] == "RON"),
    (fpa_query['Cntry_code'] == "RUB"),
    (fpa_query['Cntry_code'] == "ZAR"),
    (fpa_query['Cntry_code'] == "THB"),
    (fpa_query['Cntry_code'] == "TRY"),
    (fpa_query['Cntry_code'] == "IRN"),
    (fpa_query['Cntry_code'] == "SGD"),
    (fpa_query['Cntry_code'] == "NTN"),
    (fpa_query['Cntry_code'] == "KWN")
]

fpa_query['Currency'] = np.select(conditions, currencies, default=None)
fpa_query.sort_values(by='Currency', ascending=True, inplace=True)
fpa_query.reset_index(inplace=True)
fpa_query['FPA3M_MID']=fpa_query['FPA3M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query['FPA1M_MID']=fpa_query['FPA1M_MID']/(10**fpa_query['FWD_SCALE'])
fpa_query= fpa_query[['Currency', 'FWD_SCALE','FPA3M_MID','FPA1M_MID']]
fpa_query

,Currency,FWD_SCALE,FPA3M_MID,FPA1M_MID
0,BRL,4,0.073002,0.023680
1,CLP,0,0.950000,0.300000
2,CNY,4,-0.026666,-0.005135
3,COP,0,53.125000,20.500000
4,CZK,3,-0.056370,-0.014125
5,HUF,2,1.692750,0.555500
6,IDR,0,75.000000,26.500000
7,INR,2,0.361900,0.106700
8,KRW,0,-4.975000,-1.000000
9,MXN,4,0.319350,0.108124


In [18]:
fpa_query[['FWD_SCALE', 'Currency']]

,FWD_SCALE,Currency
0,4,BRL
1,0,CLP
2,4,CNY
3,0,COP
4,3,CZK
5,2,HUF
6,0,IDR
7,2,INR
8,0,KRW
9,4,MXN


In [30]:
end_date

'20241108'

In [6]:
fwd_scale = fpa_query[['FWD_SCALE', 'Currency']]
fwd_scale_dict = dict(zip(fwd_scale['Currency'], fwd_scale['FWD_SCALE'])) # Dictionary for efficient lookup

bquery = blp.BlpQuery().start()
fpa3m_query = bquery.bdh(
        ["BCN3M BGN Curncy","CHN3M BGN Curncy","CNH3M CMPN Curncy","CLN3M BGN Curncy","CZK3M BGN Curncy","HUF3M BGN Curncy","IHN3M CMPN Curncy","MRN3M CMPN Curncy","MXN3M BGN Curncy","PSN3M BGN Curncy","PPN3M CMPN Curncy","PLN3M BGN Curncy","RON3M BGN Curncy","RUB3M BGN Curncy","ZAR3M BGN Curncy","THB3M BGN Curncy","TRY3M BGN Curncy","IRN3M CMPN Curncy","SGD3M BGN Curncy","NTN3M BGN Curncy","KWN3M CMPN Curncy"],
        ["PX_MID"],
        start_date=strt_date,
    end_date=end_date,
    options={"adjustmentSplit": True, "CDR": "5D"} )

fpa3m_query['Cntry_code'] = fpa3m_query['security'].str[:3]
fpa3m_query.rename(columns={'PX_MID':'FPA3M_MID'}, inplace=True)

fpa1m_query = bquery.bdh(
        ["BCN1M BGN Curncy","CHN1M BGN Curncy","CNH1M CMPN Curncy","CLN1M BGN Curncy","CZK1M BGN Curncy","HUF1M BGN Curncy","IHN1M CMPN Curncy","MRN1M CMPN Curncy","MXN1M BGN Curncy","PSN1M BGN Curncy","PPN1M CMPN Curncy","PLN1M BGN Curncy","RON1M BGN Curncy","RUB1M BGN Curncy","ZAR1M BGN Curncy","THB1M BGN Curncy","TRY1M BGN Curncy","IRN1M CMPN Curncy","SGD1M BGN Curncy","NTN1M BGN Curncy","KWN1M CMPN Curncy"],
        ["PX_MID"],
        start_date=strt_date,
    end_date=end_date,
    options={"adjustmentSplit": True, "CDR": "5D"} )

fpa1m_query['Cntry_code'] = fpa1m_query['security'].str[:3]
fpa1m_query.rename(columns={'PX_MID':'FPA1M_MID'}, inplace=True)

fpa_query = pd.merge(fpa3m_query, fpa1m_query, on='Cntry_code')

conditions = [
    (fpa_query['Cntry_code'] == "BCN"),
    (fpa_query['Cntry_code'] == "CHN"),
    (fpa_query['Cntry_code'] == "CNH"),
    (fpa_query['Cntry_code'] == "CLN"),
    (fpa_query['Cntry_code'] == "CZK"),
    (fpa_query['Cntry_code'] == "HUF"),
    (fpa_query['Cntry_code'] == "IHN"),
    (fpa_query['Cntry_code'] == "MRN"),
    (fpa_query['Cntry_code'] == "MXN"),
    (fpa_query['Cntry_code'] == "PSN"),
    (fpa_query['Cntry_code'] == "PPN"),
    (fpa_query['Cntry_code'] == "PLN"),
    (fpa_query['Cntry_code'] == "RON"),
    (fpa_query['Cntry_code'] == "RUB"),
    (fpa_query['Cntry_code'] == "ZAR"),
    (fpa_query['Cntry_code'] == "THB"),
    (fpa_query['Cntry_code'] == "TRY"),
    (fpa_query['Cntry_code'] == "IRN"),
    (fpa_query['Cntry_code'] == "SGD"),
    (fpa_query['Cntry_code'] == "NTN"),
    (fpa_query['Cntry_code'] == "KWN")
]

fpa_query['Currency'] = np.select(conditions, currencies, default=None)
fpa_query.sort_values(by='Currency', ascending=True, inplace=True)
fpa_query.reset_index(inplace=True)
fpa_query['FWD_SCALE'] = fpa_query['Currency'].map(fwd_scale_dict) 
fpa_query['FPA3M_MID'] = fpa_query['FPA3M_MID'] / (10**fpa_query['FWD_SCALE'])
fpa_query['FPA1M_MID'] = fpa_query['FPA1M_MID'] / (10**fpa_query['FWD_SCALE'])

fpa_query= fpa_query[['Currency', 'FWD_SCALE','FPA3M_MID','FPA1M_MID']]
fpa_query

,Currency,FWD_SCALE,FPA3M_MID,FPA1M_MID
0,BRL,4,0.067656,0.018530
1,BRL,4,0.064277,0.020470
2,BRL,4,0.064277,0.020200
3,BRL,4,0.064277,0.019860
4,BRL,4,0.064277,0.017391
...,...,...,...,...
3019,ZAR,4,0.143290,0.048090
3020,ZAR,4,0.143290,0.051070
3021,ZAR,4,0.143290,0.052530
3022,ZAR,4,0.146820,0.048090


In [23]:
fpa3m_query

,date,security,FPA3M_MID,Cntry_code
0,2022-12-30,TRY3M BGN Curncy,13217.075,TRY
1,2022-12-30,THB3M BGN Curncy,-43.000,THB
2,2022-12-30,IRN3M CMPN Curncy,59.000,IRN
3,2022-12-30,CZK3M BGN Curncy,110.000,CZK
4,2022-12-30,MRN3M CMPN Curncy,-261.993,MRN
...,...,...,...,...
10201,2024-11-08,IRN3M CMPN Curncy,35.050,IRN
10202,2024-11-08,RUB3M BGN Curncy,32113.500,RUB
10203,2024-11-08,TRY3M BGN Curncy,32241.387,TRY
10204,2024-11-08,MXN3M BGN Curncy,3033.500,MXN


In [39]:
bquery = blp.BlpQuery().start()
IMPVOL_query = bquery.bdh(
        ["USDBRLV3M Curncy","USDCLPV3M Curncy","USDCNYV3M Curncy","USDCOPV3M Curncy","USDCZKV3M Curncy","USDHUFV3M Curncy","USDIDRV3M Curncy","USDMYRV3M Curncy","USDMXNV3M Curncy","USDPENV3M Curncy","USDPHPV3M Curncy","USDPLNV3M Curncy","USDRONV3M Curncy","USDRUBV3M Curncy","USDZARV3M Curncy","USDTHBV3M Curncy","USDTRYV3M Curncy","USDINRV3M Curncy","USDSGDV3M Curncy","USDTWDV3M Curncy","USDKRWV3M Curncy"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"}) #

IMPVOL_query.rename(columns={'date':'DATE', 'PX_LAST':'3M Implied Vol'}, inplace=True)

conditions = [
    (IMPVOL_query['security'] == 'USDBRLV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCLPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCNYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCOPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDCZKV3M Curncy'),
    (IMPVOL_query['security'] == 'USDHUFV3M Curncy'),
    (IMPVOL_query['security'] == 'USDIDRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMYRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDMXNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPENV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPHPV3M Curncy'),
    (IMPVOL_query['security'] == 'USDPLNV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRONV3M Curncy'),
    (IMPVOL_query['security'] == 'USDRUBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDZARV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTHBV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTRYV3M Curncy'),
    (IMPVOL_query['security'] == 'USDINRV3M Curncy'),
    (IMPVOL_query['security'] == 'USDSGDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDTWDV3M Curncy'),
    (IMPVOL_query['security'] == 'USDKRWV3M Curncy'),
]

IMPVOL_query['Currency'] = np.select(conditions, currencies, default=None)
IMPVOL_query.drop(columns="security", inplace=True)
IMPVOL_query.sort_values(by='DATE', ascending=False, inplace=True)
IMPVOL_query=IMPVOL_query[['DATE','Currency','3M Implied Vol']]
IMPVOL_query

,DATE,Currency,3M Implied Vol
4766,2024-11-12,MYR,6.3875
4756,2024-11-12,HUF,11.7600
4746,2024-11-12,RON,7.5850
4747,2024-11-12,PEN,4.3875
4748,2024-11-12,TWD,6.8500
...,...,...,...
17,2024-01-01,PHP,6.5325
18,2024-01-01,CLP,15.7600
19,2024-01-01,PEN,7.4225
20,2024-01-01,CNY,4.5000


In [84]:
currencies = ["BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN", "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD", "KRW"]

bquery = blp.BlpQuery().start()

bquery = blp.BlpQuery().start()
GDP_query = bquery.bdh(
        ["WGDPBRAZ Index","WGDPCHIL Index","WGDPCHIN Index","WGDPCOLO Index","WGDPCZEC Index","WGDPHUNG Index","WGDPINDO Index","WGDPMALY Index","WGDPMEXI Index","WGDPPERU Index","WGDPPHIL Index","WGDPPOLA Index","WGDPROMA Index","WGDPRUSS Index","WGDPSOUT Index","WGDPTHAI Index","WGDPTURK Index","WGDPINDI Index","WGDPSING Index","TAGDPARE Index","WGDPKORE Index"],
        ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"}) #)

GDP_query.rename(columns={'PX_LAST':'GDP'}, inplace=True)

conditions = [
    (GDP_query['security'] == 'WGDPBRAZ Index'),
    (GDP_query['security'] == 'WGDPCHIL Index'),
    (GDP_query['security'] == 'WGDPCHIN Index'),
    (GDP_query['security'] == 'WGDPCOLO Index'),
    (GDP_query['security'] == 'WGDPCZEC Index'),
    (GDP_query['security'] == 'WGDPHUNG Index'),
    (GDP_query['security'] == 'WGDPINDO Index'),
    (GDP_query['security'] == 'WGDPMALY Index'),
    (GDP_query['security'] == 'WGDPMEXI Index'),
    (GDP_query['security'] == 'WGDPPERU Index'),
    (GDP_query['security'] == 'WGDPPHIL Index'),
    (GDP_query['security'] == 'WGDPPOLA Index'),
    (GDP_query['security'] == 'WGDPROMA Index'),
    (GDP_query['security'] == 'WGDPRUSS Index'),
    (GDP_query['security'] == 'WGDPSOUT Index'),
    (GDP_query['security'] == 'WGDPTHAI Index'),
    (GDP_query['security'] == 'WGDPTURK Index'),
    (GDP_query['security'] == 'WGDPINDI Index'),
    (GDP_query['security'] == 'WGDPSING Index'),
    (GDP_query['security'] == 'TAGDPARE Index'),
    (GDP_query['security'] == 'WGDPKORE Index')
]

GDP_query['Currency'] = np.select(conditions, currencies, default=None)
GDP_query.sort_values(by='Currency', ascending=True, inplace=True)
GDP_query.reset_index(inplace=True)
GDP_query=GDP_query[['Currency','GDP']]
GDP_query

,Currency,GDP
0,BRL,None
1,BRL,None
2,BRL,None
3,BRL,None
4,BRL,None
...,...,...
205,ZAR,None
206,ZAR,None
207,ZAR,None
208,ZAR,None


In [76]:
bquery = blp.BlpQuery().start()

###########################################################################################################################################################
###########################################################################################################################################################
bquery = blp.BlpQuery().start()
trade_balance_tickers = [
    "ECOYBBRN Index", "ECOYBCLN Index", "ECOYBCNN Index", "ECOYBCON Index",
    "ECOYBCZN Index", "ECOYBHUN Index", "ECOYBIDN Index", "ECOYBMYN Index",
    "ECOYBMXN Index", "PRTRBAL Index", "ECOYBPHN Index", "ECOYBPLN Index",
    "ECOYBRON Index", "ECOYBRUN Index", "ECOYBZAN Index", "ECOYBTHN Index",
    "ECOYBTRN Index", "ECOYBINN Index", "ECOYBSGN Index", "ECOYBTWN Index",
    "ECOYBKRN Index", "ECOYBEGN Index","ECOYBEGN Index", "ECOYBEAS Index", "ECOYBJPN Index" 
]

currencies = [
    "BRL", "CLP", "CNY", "COP", "CZK", "HUF", "IDR", "MYR", "MXN", "PEN",
    "PHP", "PLN", "RON", "RUB", "ZAR", "THB", "TRY", "INR", "SGD", "TWD",
    "KRW", "UAH", "EGP", "EUR", "JPY"
]

TB_query = bquery.bdh(trade_balance_tickers, ["PX_LAST"],
        start_date= strt_date, 
        end_date= end_date, 
        options={"adjustmentSplit": True, "CDR":"5D"})
TB_query.rename(columns={'PX_LAST':'TB'}, inplace=True)

# Create conditions dynamically
conditions = [(TB_query['security'] == ticker) for ticker in trade_balance_tickers]

TB_query['Currency'] = np.select(conditions, currencies, default=None) 

C:\Program64\Python\Python\lib\site-packages\blp\blp.py:775: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pandas.concat(dfs)


In [83]:
def get_3m_chg(row):  # Now takes a row as input
    security = row['security']
    date = row['date']  # Get the date from the row
    
    # Calculate EOM dates based on the row's date
    EOM_LAST_UPDATE = (date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
    EOM_LAST_UPDATE_3M = (date - pd.DateOffset(months=4)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)

    start_date = EOM_LAST_UPDATE_3M.strftime('%Y-%m-%d')
    end_date = EOM_LAST_UPDATE.strftime('%Y-%m-%d')
    
    try: #Use try except for error handling
        query_text = f"get(NET_CHG(px_last(Dates=range({start_date}, {end_date}, Frq=M), Fill=PREV))) for(['{security}'])"
        BQL_query = bquery.bql(expression=f"{query_text}")
        return BQL_query.loc[0, 'value'] #Bloomberg returns the value in the first row

    except Exception as e: # Handle exceptions during Bloomberg query
        print(f"Error for {security} on {date}: {e}")
        return None  # Or some other appropriate default value
TB_query['3m_Chg'] = TB_query.apply(get_3m_chg, axis=1)


In [81]:
TB_query

,date,security,TB,Currency,3m_Chg
0,2024-10-30,ECOYBJPN Index,-41.8,JPY,-0.8
1,2024-10-30,ECOYBINN Index,-261.4,INR,-6.3
2,2024-10-30,ECOYBTRN Index,-78.5,TRY,3.8
3,2024-10-30,ECOYBRON Index,-34.7,RON,-0.8
4,2024-10-30,ECOYBSGN Index,49.9,SGD,1.2
...,...,...,...,...,...
245,2024-11-12,PRTRBAL Index,NaN,PEN,0.0
246,2024-11-12,ECOYBSGN Index,49.9,SGD,-0.1
247,2024-11-12,ECOYBTWN Index,87.5,TWD,-2.1
248,2024-11-12,ECOYBHUN Index,12.8,HUF,-0.2


In [80]:
### VERSION WITH CURRENT DATE WITHOUT LAST UPDATE 
def get_3m_chg(row):
    security = row['security']
    date = row['date']  # Takes security ticker as input
    try:
        LAST_UPDATE = bquery.bdp([f"{security}"], ["LAST_UPDATE_DT"]).loc[0, 'LAST_UPDATE_DT']
        EOM_LAST_UPDATE = (date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        EOM_LAST_UPDATE_3M = (date - pd.DateOffset(months=4)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        start_date = EOM_LAST_UPDATE_3M.strftime('%Y-%m-%d')
        end_date = EOM_LAST_UPDATE.strftime('%Y-%m-%d')
        query_text = f"get(NET_CHG(px_last(Dates=range({start_date}, {end_date}, Frq=M), Fill=PREV))) for(['{security}'])"
        BQL_query = bquery.bql(expression=f"{query_text}")
        return BQL_query.loc[0, 'value']  # Corrected index to 0

    except Exception as e:
        print(f"Error for {security}: {e}")
        return None
TB_query['3m_Chg'] = TB_query.apply(get_3m_chg, axis=1)


In [74]:
unique_securities = TB_query['security'].unique()
three_month_changes = {security: get_3m_chg(security) for security in unique_securities}
TB_query['TB_3m_Chg'] = TB_query['security'].map(three_month_changes)
print(TB_query)


          date        security     TB Currency  TB_3m_Chg
0   2024-10-30  ECOYBJPN Index  -41.8      JPY       -4.4
1   2024-10-30  ECOYBINN Index -261.4      INR       -9.1
2   2024-10-30  ECOYBTRN Index  -78.5      TRY        9.0
3   2024-10-30  ECOYBCZN Index    9.9      CZK        1.3
4   2024-10-30  ECOYBSGN Index   49.9      SGD        1.6
..         ...             ...    ...      ...        ...
245 2024-11-12  ECOYBCLN Index   21.4      CLP        2.4
246 2024-11-12  ECOYBSGN Index   49.9      SGD        1.6
247 2024-11-12  ECOYBTWN Index   87.5      TWD        0.8
248 2024-11-12  ECOYBRUN Index    NaN      RUB       41.2
249 2024-11-12  ECOYBEAS Index    NaN      EUR       20.9

[250 rows x 5 columns]


In [75]:

def get_3m_chg(row):
    security = row['security']
    date = row['date']

    try:
        # 1. Get historical LAST_UPDATE_DT (this is the key change)
        historical_last_update = bquery.bdh(
            [f"{security}"],  # Ticker
            "LAST_UPDATE_DT", # Field
            date,         # Start date (the date from the row)
            date,         # End date (same as start date to get a single value)
            #Other overrides that might be necessary for the query. For example:
            # overrides =  [("Product","CN")]            
        )

        #Check if a result is given for the given ticker and date, otherwise result will be None
        if historical_last_update.empty:
            return None

        #Extract the last update date for the ticker, for the given date. The result is in the second level of the multiIndex columns
        historical_last_update = historical_last_update.columns.get_level_values(1)[0]


        # 2. Calculate dates relative to the *historical* LAST_UPDATE_DT
        eom_last_update = (historical_last_update - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        eom_last_update_3m = (historical_last_update - pd.DateOffset(months=4)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)


        start_date = eom_last_update_3m.strftime('%Y-%m-%d')
        end_date = eom_last_update.strftime('%Y-%m-%d')

        # 3. BQL query (now using dates based on historical LAST_UPDATE_DT)
        query_text = f"get(NET_CHG(px_last(Dates=range({start_date}, {end_date}, Frq=M), Fill=PREV))) for(['{security}'])"
        bql_query = bquery.bql(expression=f"{query_text}")
        
        #Check if the Bloomberg query returns a result
        if (bql_query.empty):
            return None
        else:
            return bql_query.loc[0, 'value']

    except Exception as e:
        print(f"Error for {security} on {date}: {e}")
        return None


In [77]:
TB_query['3m_Chg'] = TB_query.apply(get_3m_chg, axis=1)


Error for ECOYBJPN Index on 2024-10-30 00:00:00: Response contains responseError
response: {'eventType': 5, 'eventTypeName': 'blpapi.Event.RESPONSE', 'messageNumber': 0, 'message': {'fragmentType': 0, 'correlationIds': [6868], 'messageType': 'HistoricalDataResponse', 'timeReceived': Timestamp('2024-11-13 00:49:14.639388+0000', tz='UTC'), 'element': {'HistoricalDataResponse': {'responseError': {'source': 'rsfhdsvc3', 'code': 30, 'category': 'BAD_ARGS', 'message': 'Invalid start date specified [nid:52892] ', 'subcategory': 'INVALID_START_DATE'}}}}}
request: {'HistoricalDataRequest': {'securities': ['ECOYBJPN Index'], 'fields': ['LAST_UPDATE_DT'], 'startDate': Timestamp('2024-10-30 00:00:00'), 'endDate': Timestamp('2024-10-30 00:00:00')}}
Error for ECOYBINN Index on 2024-10-30 00:00:00: Response contains responseError
response: {'eventType': 5, 'eventTypeName': 'blpapi.Event.RESPONSE', 'messageNumber': 0, 'message': {'fragmentType': 0, 'correlationIds': [6869], 'messageType': 'HistoricalD

In [63]:

def get_peru_3m_chg(date):
    security = "PRTRBAL Index"
    try:
        # Calculate end-of-month dates relative to the input 'date'
        EOM_date = (date - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
        EOM_date_3m_ago = (date - pd.DateOffset(months=3)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)

        # Calculate start and end dates for current and previous year periods
        start_date_current = EOM_date_3m_ago.strftime('%Y-%m-%d')
        end_date_current = EOM_date.strftime('%Y-%m-%d')
        start_date_previous = (EOM_date_3m_ago - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
        end_date_previous = (EOM_date - pd.DateOffset(months=12)).strftime('%Y-%m-%d')

        # Bloomberg queries
        BQL_query_current = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_current}, {end_date_current})))) for(['{security}'])")
        BQL_query_previous = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_previous}, {end_date_previous})))) for(['{security}'])")


        #Check if both Bloomberg Queries provide a result, otherwise 3m change will be None
        if (BQL_query_current.empty or BQL_query_previous.empty):
             return None
        else:
            three_month_change = (BQL_query_current.loc[0, 'value'] - BQL_query_previous.loc[0, 'value']) / 1000
            return three_month_change

    except Exception as e:
        print(f"Error for {security} on {date}: {e}")
        return None



start_date = pd.to_datetime('2024-01-01') # Replace with your actual start date
end_date = pd.to_datetime('2024-11-12') # Replace with your actual end date

# Generate a date range
date_range = pd.date_range(start=start_date, end=end_date, freq='B') # 'B' for business days


# Create a DataFrame to store results
peru_trade_balance = pd.DataFrame({'date': date_range})

# Calculate the 3-month change for each date
peru_trade_balance['TB_3m_Chg'] = peru_trade_balance['date'].apply(get_peru_3m_chg)

# ... (Now you can merge peru_trade_balance with your main DataFrame if needed) ...
print(peru_trade_balance)

In [66]:

bquery = blp.BlpQuery().start()

def get_3m_chg(df):
    security = df['security']
    LAST_UPDATE = bquery.bdp( [f"{security}"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
    EOM_LAST_UPDATE = (LAST_UPDATE - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
    EOM_LAST_UPDATE_3M = (LAST_UPDATE - pd.DateOffset(months=4)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
    start_date = EOM_LAST_UPDATE_3M.strftime('%Y-%m-%d')
    end_date = EOM_LAST_UPDATE.strftime('%Y-%m-%d')
    query_text = f"get(NET_CHG(px_last(Dates=range({start_date}, {end_date}, Frq=M), Fill=PREV))) for(['{security}'])"
    BQL_query = bquery.bql(expression=f"{query_text}")
    return BQL_query.loc[1,'value']

TB_query['TB_3m_Chg'] = TB_query.apply(get_3m_chg, axis=1)
TB_query.sort_values(by='Currency', ascending=True, inplace=True)
TB_query.reset_index(inplace=True)
TB_query=TB_query[['Currency', 'security', 'TB', 'TB_3m_Chg']]

###SPECIAL CASE FOR PERU
security = "PRTRBAL Index"
LAST_UPDATE = bquery.bdp( [f"{security}"],["LAST_UPDATE_DT"]).loc[0,'LAST_UPDATE_DT']
EOM_LAST_UPDATE = (LAST_UPDATE - pd.DateOffset(months=1)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
EOM_LAST_UPDATE_3M = (LAST_UPDATE - pd.DateOffset(months=3)).replace(day=1) + pd.DateOffset(months=2) - pd.DateOffset(days=1)
start_date_1 = EOM_LAST_UPDATE_3M.strftime('%Y-%m-%d')
end_date_1 = EOM_LAST_UPDATE.strftime('%Y-%m-%d')
start_date_2 = (EOM_LAST_UPDATE_3M - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
end_date_2 = (EOM_LAST_UPDATE - pd.DateOffset(months=12)).strftime('%Y-%m-%d')
BQL_query_1 = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_1}, {end_date_1})))) for(['{security}'])")
BQL_query_2 = bquery.bql(expression=f"get(SUM(px_last(Dates=range({start_date_2}, {end_date_2})))) for(['{security}'])")

TB_query.loc[TB_query['Currency'] == 'PEN', 'TB_3m_Chg'] = (BQL_query_1.loc[1,'value']-BQL_query_2.loc[1,'value'])/1000


TBoverGDP = pd.merge(TB_query, GDP_query, on='Currency')
# Find the index of the row where Currency is 'BRL'
brl_index = TBoverGDP[TBoverGDP['Currency'] == 'BRL'].index[0]
################# CUSTOM FOR BRL AND TWD SEE LATER ###########################################################################
###########################################################################

# Divide the TB_3m_Chg_GDP value for 'BRL' by 5.5
TBoverGDP.loc[brl_index, 'TB_3m_Chg'] = -12
# converting to usd for TWD 
twd_index = TBoverGDP[TBoverGDP['Currency'] == 'TWD'].index[0]
TBoverGDP.loc[twd_index, 'GDP'] /= 30.817 # CONVERT TO DOLLARS !! 


TBoverGDP["TB_3m_Chg_GDP"] = TBoverGDP["TB_3m_Chg"]/TBoverGDP["GDP"]
print(TBoverGDP[["GDP",'Currency','TB_3m_Chg']])
TBoverGDP=TBoverGDP[['Currency', 'TB_3m_Chg_GDP']]

# we convert to dollars for BRL 


TBoverGDP

            GDP Currency  TB_3m_Chg
0           NaN      BRL      -12.0
1      2173.666      BRL      -14.1
2      2173.666      BRL      -14.1
3      2173.666      BRL      -14.1
4      2173.666      BRL      -14.1
...         ...      ...        ...
47875   377.782      ZAR       -0.2
47876   377.782      ZAR       -0.2
47877   377.782      ZAR       -0.2
47878   377.782      ZAR       -0.2
47879   377.782      ZAR       -0.2

[47880 rows x 3 columns]


,Currency,TB_3m_Chg_GDP
0,BRL,NaN
1,BRL,-0.006487
2,BRL,-0.006487
3,BRL,-0.006487
4,BRL,-0.006487
...,...,...
47875,ZAR,-0.000529
47876,ZAR,-0.000529
47877,ZAR,-0.000529
47878,ZAR,-0.000529


In [68]:
LAST_UPDATE

Timestamp('2024-08-31 00:00:00')

In [67]:
BQL_query_2

,security,field,secondary_name,secondary_value,value
0,PRTRBAL Index,"SUM(px_last(Dates=range(2023-06-30,2023-08-31)))",CURRENCY,USD,3744.839928
1,PRTRBAL Index,"SUM(px_last(Dates=range(2023-06-30,2023-08-31)))",DATE,2023-08-31T00:00:00Z,3744.839928


In [54]:
data_file = pd.ExcelFile('Metrics data.xlsx')
metrics = pd.read_excel(data_file, 'Metrics',header=0)
metrics = metrics.rename(columns={'Unnamed: 0': 'metric'})

In [41]:
metrics

,metric,BRL,CLP,CNY,COP,CZK,HUF,IDR,MYR,MXN,...,TRY,INR,SGD,TWD,KRW,UAH,EGP,EUR,JPY,Avg/Std
0,2021-03-18 00:00:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,Momentum,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,NaN
2,REER discount,-0.250451,-0.031295,0.053956,-0.089563,0.064574,-0.036405,-0.018065,-0.021672,0.005603,...,-0.139202,-0.009631,-0.027025,0.031661,-0.015888,0.060914,0.110721,0.014202,-0.038838,NaN
3,Conditional REER,-0.250451,0.000000,0.000000,-0.089563,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-0.009631,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058682
4,3m chg in TB %GDP,0.002699,0.032814,0.012307,-0.001105,0.016712,0.017329,0.013511,0.034717,0.017040,...,-0.007778,0.003195,0.023235,0.000796,0.007143,0.002463,-0.001369,0.002934,0.005457,0.013178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,Conditional REER,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
2044,3m chg in TB %GDP,-0.001236,0.007749,0.003956,-0.000275,0.003929,-0.004238,-0.001094,-0.017515,-0.001174,...,0.008123,-0.000845,0.000798,0.001112,0.004087,-0.011021,-0.008840,0.001055,-0.000973,0.006084
2045,Nominal Carry,0.047685,0.003457,-0.009569,0.045088,-0.008943,0.016597,0.016609,-0.015515,0.056870,...,0.371424,0.015297,-0.014292,-0.027122,-0.013682,0.232978,0.180465,-0.015194,-0.043260,0.082164
2046,Real Carry,0.026573,-0.016982,0.013389,0.017551,-0.011075,0.010887,0.025983,-0.007925,0.035737,...,-0.053974,-0.019573,-0.008653,-0.018762,-0.001043,0.156050,-0.043361,-0.009578,-0.043120,0.020172


In [55]:
model_list = pd.read_excel(data_file, 'Model list',header=0)

In [56]:
model_list

,model,Date
0,SN-Agn,2024-11-13
1,SN-Agn,2024-11-12
2,RN-Awa,2024-11-11
3,RN-Awa,2024-11-08
4,CN-Agn,2024-11-07
...,...,...
223,SN-Awa,2024-01-05
224,CR-Agn,2024-01-04
225,RR-Awa,2024-01-03
226,SN-Awa,2024-01-02


In [57]:
def create_date_metric_dict(metrics_df):
    """Creates a dictionary with dates as keys and metric DataFrames as values."""

    date_metric_dict = {}
    i = 0
    while i < len(metrics_df):
        try:
            current_date = pd.to_datetime(metrics_df.iloc[i, 0]).strftime('%Y-%m-%d') 
            metrics_for_date = metrics_df.iloc[i+1:i+8].copy()  
            metrics_for_date = metrics_for_date.set_index('metric') 
            metrics_for_date = metrics_for_date.drop(columns=[c for c in metrics_for_date.columns if c.startswith("Unnamed")])

            date_metric_dict[current_date] = metrics_for_date.transpose()
            i += 7

        except (ValueError, TypeError, IndexError):
            print(f"Skipping invalid row at index {i}")
            i += 1
    return date_metric_dict

date_metric_dictionary = create_date_metric_dict(metrics)


Skipping invalid row at index 7
Skipping invalid row at index 15
Skipping invalid row at index 23
Skipping invalid row at index 31
Skipping invalid row at index 39
Skipping invalid row at index 47
Skipping invalid row at index 55
Skipping invalid row at index 63
Skipping invalid row at index 71
Skipping invalid row at index 79
Skipping invalid row at index 87
Skipping invalid row at index 95
Skipping invalid row at index 103
Skipping invalid row at index 111
Skipping invalid row at index 119
Skipping invalid row at index 127
Skipping invalid row at index 135
Skipping invalid row at index 143
Skipping invalid row at index 151
Skipping invalid row at index 159
Skipping invalid row at index 167
Skipping invalid row at index 175
Skipping invalid row at index 183
Skipping invalid row at index 191
Skipping invalid row at index 199
Skipping invalid row at index 207
Skipping invalid row at index 215
Skipping invalid row at index 223
Skipping invalid row at index 231
Skipping invalid row at ind

In [58]:

def process_metric_dataframes(date_metric_dict):
    """Renames columns and adjusts '3M Implied Vol' within each DataFrame."""
    rows_to_drop = ['UAH', 'EGP', 'EUR', 'JPY', 'RUB']
    columns_to_drop_if_present = ['Conditional_REER_Discount', 'Symmetrical_REER_Discount']  # Columns to drop conditionally

    for date, df in date_metric_dict.items():
        column_mapping = {  # Mapping for column renaming
            'metric' : 'Currency',
            'REER discount': 'REER_Discount_x',
            'Symmetrical REER': 'Symmetrical_REER_Discount',
            '3m chg in TB %GDP': 'TB_3m_Chg_GDP',
            'Nominal Carry': 'Nominal_carry',
            'Real Carry': 'Real_Carry',
            'Implied Volatility (%)': '3M Implied Vol',
            'Conditional REER': 'Conditional_REER_Discount'
        }
        
        df = df.rename(columns=column_mapping, errors='ignore')
        for row_label in rows_to_drop:
            try:  # Handle potential KeyError if label doesn't exist
                df.drop(row_label, axis=0, inplace=True)
            except KeyError:
                print(f"Row '{row_label}' not found in DataFrame for date {date}. Skipping.")  # Informative message

        for col in columns_to_drop_if_present:
            if col in df.columns:
                df.drop(col, axis=1, inplace=True)
                



        if '3M Implied Vol' in df.columns:
            df['3M Implied Vol'] = df['3M Implied Vol'] / 100  



        date_metric_dict[date] = df  


    return date_metric_dict  



def process_metric_dataframes2(date_metric_dict):
    """Renames columns and adjusts '3M Implied Vol' within each DataFrame."""



    for date, df in date_metric_dict.items():


        df_new = df.copy() # Create a copy to avoid modifying the original DataFrame

        df_new = df_new.reset_index(drop=False)  # Reset index
        df_new.columns = ['Currency'] + list(df_new.columns[1:]) #Rename the first column

        date_metric_dict[date] = df_new 
    return date_metric_dict  

In [59]:
date_metric_dictionary = create_date_metric_dict(metrics)
date_metric_dictionary = process_metric_dataframes(date_metric_dictionary) 
date_metric_dictionary = process_metric_dataframes2(date_metric_dictionary) 

Skipping invalid row at index 7
Skipping invalid row at index 15
Skipping invalid row at index 23
Skipping invalid row at index 31
Skipping invalid row at index 39
Skipping invalid row at index 47
Skipping invalid row at index 55
Skipping invalid row at index 63
Skipping invalid row at index 71
Skipping invalid row at index 79
Skipping invalid row at index 87
Skipping invalid row at index 95
Skipping invalid row at index 103
Skipping invalid row at index 111
Skipping invalid row at index 119
Skipping invalid row at index 127
Skipping invalid row at index 135
Skipping invalid row at index 143
Skipping invalid row at index 151
Skipping invalid row at index 159
Skipping invalid row at index 167
Skipping invalid row at index 175
Skipping invalid row at index 183
Skipping invalid row at index 191
Skipping invalid row at index 199
Skipping invalid row at index 207
Skipping invalid row at index 215
Skipping invalid row at index 223
Skipping invalid row at index 231
Skipping invalid row at ind

In [60]:

desired_order = ['BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HUF', 'IDR', 'INR', 'KRW', 'MXN', 'MYR', 'PEN', 'PHP', 'PLN', 'RON', 'SGD', 'THB', 'TRY', 'TWD', 'ZAR', 'Avg/Std']


def reorder_currencies(df, desired_order=desired_order):

    # Separate currency rows and 'Avg/Std' row
    currency_rows = df[df['Currency'] != 'Avg/Std']
    avg_std_row = df[df['Currency'] == 'Avg/Std']
    
    # Create a categorical type for the Currency column with the desired order
    currency_rows['Currency'] = pd.Categorical(currency_rows['Currency'], categories=desired_order, ordered=True)
    currency_rows_reordered = currency_rows.sort_values('Currency')
    
    currency_rows_reordered['Currency'] = currency_rows_reordered['Currency'].astype(str) # convert back to string if you need
    
    # Concatenate reordered rows and 'Avg/Std' row
    reordered_df = pd.concat([currency_rows_reordered, avg_std_row], ignore_index=True)
    
    return reordered_df

# Apply the function to each DataFrame
for date, df in date_metric_dictionary.items():
    date_metric_dictionary[date] = reorder_currencies(df, desired_order)


C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_19336/3238837231.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_rows['Currency'] = pd.Categorical(currency_rows['Currency'], categories=desired_order, ordered=True)
C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_19336/3238837231.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  currency_rows['Currency'] = pd.Categorical(currency_rows['Currency'], categories=desired_order, ordered=True)
C:\Users\UT3P5T\AppData\Local\Temp\1/ipykernel_19336/32388

In [71]:
date_metric_dictionary['2024-11-12']

,Currency,Momentum,REER_Discount_x,TB_3m_Chg_GDP,Nominal_carry,Real_Carry,3M Implied Vol,Symmetrical_REER_Discount,Conditional_REER_Discount,z_score,conviction_SN-Agn
0,BRL,1.0,-0.059093,-0.000564,0.042643,0.019552,0.152150,-0.059093,-0.059093,0.570700,3.0
1,CLP,-1.0,-0.072838,0.007153,0.003765,-0.018665,0.138000,-0.000000,0.000000,0.399773,2.0
2,CNY,-1.0,-0.063990,0.003945,-0.016489,0.004188,0.062325,-0.000000,0.000000,0.140159,1.0
3,COP,-1.0,0.036060,-0.000275,0.043424,0.013911,0.139175,0.036060,0.000000,-0.081002,0.0
4,CZK,-1.0,0.063654,0.003929,-0.009070,-0.013194,0.086650,0.063654,0.000000,-0.277189,-1.0
5,HUF,-1.0,0.015728,-0.004238,0.017046,0.009352,0.117600,0.015728,0.000000,-0.264274,-1.0
6,IDR,-1.0,-0.007079,-0.000875,0.016209,0.023582,0.073500,-0.000000,0.000000,0.023208,0.0
7,INR,-1.0,0.025557,-0.002563,0.014543,-0.022304,0.026810,0.025557,0.000000,-0.254622,0.0
8,KRW,-1.0,-0.063878,0.004087,-0.014507,-0.003885,0.091500,-0.000000,0.000000,0.156343,2.0
9,MXN,-1.0,0.016279,-0.001174,0.056218,0.033097,0.153250,0.016279,0.000000,0.065575,1.0


In [62]:
for date, df in date_metric_dictionary.items():
    df['Momentum'] = df['Momentum'].replace(0, -1)  # Replace 0 with -1 in 'Momentum'
    date_metric_dictionary[date] = df  # Update the dictionary

In [ ]:
#COND and SYM REER DISCOUNT 
REER_discount = pd.merge(REER, Momentum, on='Currency')
REER_discount = REER_discount[['Currency', 'REER_Discount', 'Momentum']]
REER_discount['Conditional_REER_Discount'] = REER_discount.apply(lambda row: row['Momentum'] * row['REER_Discount'] * (row['REER_Discount'] < 0) if row['Momentum'] == 1 else 0, axis=1)
REER_discount['Symmetrical_REER_Discount'] = REER_discount.apply(
    lambda row: row['REER_Discount'] * (np.sign(row['Momentum']) * np.sign(row['REER_Discount']) < 0  ), 
    axis=1
)
REER_discount 

In [63]:
def calculate_symmetrical_reer(df):
    """Calculates Symmetrical REER Discount based on the Excel logic."""

    df['Symmetrical_REER_Discount'] = df.apply(
        lambda row: row['REER_Discount_x'] * (
            1 if (row['Momentum'] == 1 and row['REER_Discount_x'] < 0) or (row['Momentum'] == -1 and row['REER_Discount_x'] > 0)
            else 0
        ),
        axis=1
    )

    
    # Calculate average standard deviation for currencies (excluding 'Avg/Std' row)
    currencies_df = df[df['Currency'] != 'Avg/Std']
    avg_std = currencies_df['Symmetrical_REER_Discount'].std()

    # Update 'Avg/Std' row
    df.loc[df['Currency'] == 'Avg/Std', 'Symmetrical_REER_Discount'] = avg_std

    return df

for date, df in date_metric_dictionary.items():
    date_metric_dictionary[date] = calculate_symmetrical_reer(df)

In [64]:
def add_conditional_reer(df):
    """
    Adds 'Conditional_REER_Discount' column if it doesn't exist.
    """
    if 'Conditional_REER_Discount' not in df.columns:
        df['Conditional_REER_Discount'] = df.apply(
            lambda row: row['Momentum'] * row['REER_Discount_x'] * (row['REER_Discount_x'] < 0) if row['Momentum'] == 1 else 0, 
            axis=1
        )
    return df


# Apply the function to each DataFrame in the dictionary
for date, df in date_metric_dictionary.items():
    date_metric_dictionary[date] = add_conditional_reer(df)

In [65]:
### ADAPTED Z SCORE FUNCTION #######
models = {  # Dictionary mapping short model names to criteria lists
    "RN": ["REER_Discount_x", "TB_3m_Chg_GDP", "Nominal_carry"],
    "CN": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "SN": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "RR": ["REER_Discount_x", "TB_3m_Chg_GDP", "Real_Carry"],
    "CR": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
    "SR": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
}
def calculate_zscore(df, criteria):
    """Calculates the z-score based on the provided criteria."""
    zscore_parts = []

    for i, criterion in enumerate(criteria):
        if df[criterion].std() == 0:  # Handle zero standard deviation
            zscore_part = 0
        elif criterion in ["Symmetrical_REER_Discount", "Conditional_REER_Discount"]:
            zscore_part = -df[criterion] / df[criterion].std()
        else:
            zscore_part = df[criterion] / df[criterion].std()

        zscore_parts.append(zscore_part)

    # Return NA for the 'Avg/Std' row
    return np.where(df['Currency'] == 'Avg/Std', np.nan, sum(zscore_parts)/3)


# Loop through each date in the dictionary
for date, df in date_metric_dictionary.items():
    print(date)
    # Find the recommended model for the current date
    model_row = model_list[model_list['Date'] == date]
    
    if not model_row.empty:
        model_name = model_row['model'].iloc[0][:2]  # Extract first two characters (e.g., "SN", "RN")
        criteria = models[model_name]  # Get criteria for the model


        df['z_score'] = calculate_zscore(df, criteria)



2021-03-18
2021-04-01
2021-05-03
2021-06-01
2021-07-01
2021-08-02
2021-09-01
2021-10-01
2021-11-01
2021-12-01
2022-01-03
2022-02-01
2022-03-01
2022-04-01
2022-05-02
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-03
2022-11-01
2022-12-01
2023-01-03
2023-02-01
2023-03-01
2023-04-03
2023-05-01
2023-06-01
2023-07-03
2023-08-01
2023-09-01
2023-10-02
2023-11-01
2023-12-01
2024-01-02
2024-01-03
2024-01-04
2024-01-05
2024-01-08
2024-01-09
2024-01-10
2024-01-11
2024-01-12
2024-01-15
2024-01-16
2024-01-17
2024-01-18
2024-01-19
2024-01-22
2024-01-23
2024-01-24
2024-01-25
2024-01-26
2024-01-29
2024-01-30
2024-01-31
2024-02-01
2024-02-02
2024-02-05
2024-02-06
2024-02-07
2024-02-08
2024-02-09
2024-02-12
2024-02-13
2024-02-14
2024-02-15
2024-02-16
2024-02-19
2024-02-20
2024-02-21
2024-02-22
2024-02-23
2024-02-26
2024-02-27
2024-02-29
2024-03-01
2024-03-04
2024-03-05
2024-03-06
2024-03-07
2024-03-08
2024-03-11
2024-03-12
2024-03-13
2024-03-14
2024-03-15
2024-03-18
2024-03-19
2024-03-20
2024-03-21

In [66]:
model_list['Date'].values

array(['2024-11-13T00:00:00.000000000', '2024-11-12T00:00:00.000000000',
       '2024-11-11T00:00:00.000000000', '2024-11-08T00:00:00.000000000',
       '2024-11-07T00:00:00.000000000', '2024-11-06T00:00:00.000000000',
       '2024-11-05T00:00:00.000000000', '2024-11-04T00:00:00.000000000',
       '2024-11-01T00:00:00.000000000', '2024-10-31T00:00:00.000000000',
       '2024-10-30T00:00:00.000000000', '2024-10-29T00:00:00.000000000',
       '2024-10-28T00:00:00.000000000', '2024-10-25T00:00:00.000000000',
       '2024-10-24T00:00:00.000000000', '2024-10-23T00:00:00.000000000',
       '2024-10-22T00:00:00.000000000', '2024-10-21T00:00:00.000000000',
       '2024-10-18T00:00:00.000000000', '2024-10-17T00:00:00.000000000',
       '2024-10-16T00:00:00.000000000', '2024-10-15T00:00:00.000000000',
       '2024-10-14T00:00:00.000000000', '2024-10-11T00:00:00.000000000',
       '2024-10-10T00:00:00.000000000', '2024-10-09T00:00:00.000000000',
       '2024-10-08T00:00:00.000000000', '2024-10-07

In [68]:
import pandas as pd
import numpy as np
models = {
    "RN": ["REER_Discount_x", "TB_3m_Chg_GDP", "Nominal_carry"],
    "CN": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "SN": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Nominal_carry"],
    "RR": ["REER_Discount_x", "TB_3m_Chg_GDP", "Real_Carry"],
    "CR": ["Conditional_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
    "SR": ["Symmetrical_REER_Discount", "TB_3m_Chg_GDP", "Real_Carry"],
}

def round_2x_to_half(x):
    two_x = 2 * x
    return round(two_x * 2) / 2 if two_x >= 0 else -np.round(-two_x * 2) / 2

# Conviction mapping dictionaries (define outside the function)
conviction_mapping_agnostic = {  # No change here
    0: 3, 1: 3, 2: 2, 3: 2, 4: 2, 5: 1, 6: 1, 7: 1,
    8: 0, 9: 0, 10: 0, 11: 0, 12: -1, 13: -1, 14: -1,
    15: -2, 16: -2, 17: -2, 18: -3, 19: -3
}


def calculate_conviction_and_zscore(df, model_name):
    """Calculates both conviction and z-score, handling 'Avg/Std' row and NaNs."""

    model_type = model_name.split("-")[1]
    short_model_name = model_name[:2]
    criteria = models.get(short_model_name)

    if criteria:
        # Filter out "Avg/Std" *before* calculations:
        df_currencies = df[df['Currency'] != 'Avg/Std'].copy()  #Important: create a copy to avoid SettingWithCopyWarning



        # 1. Calculate Z-score:
        df_currencies[f'zscore_{short_model_name}'] = calculate_zscore(df_currencies, criteria)

        # 2. Calculate Rank (and fill NaNs if any):
        df_currencies[f'rank_{short_model_name}'] = df_currencies[f'zscore_{short_model_name}'].rank(ascending=False, method='dense').fillna(-1).astype(int)


        # 3. Calculate Conviction:
        if model_type == 'Agn':
            df_sorted = df_currencies.sort_values(by=f'rank_{short_model_name}')

            df_sorted[f'conviction_{model_name}'] = df_sorted[f'rank_{short_model_name}'].apply(
                 lambda rank: conviction_mapping_agnostic.get(int(rank) - 1, np.nan) if not pd.isna(rank) else None)

            df = df.merge(df_sorted[[f'conviction_{model_name}', 'Currency']], on='Currency', how='left') #Important: add currency to merge on

        elif model_type == 'Awa':

            df_currencies[f'conviction_{model_name}'] = df_currencies.apply(
                lambda row: _calculate_awa_conviction(row, short_model_name), axis=1
             )
            df = df.merge(df_currencies[[f'conviction_{model_name}', 'Currency']], on='Currency', how='left')#Merge the result back


    return df

def _calculate_awa_conviction(row, model_name): #Correct the ranges 
    try:
        rank = int(row[f'rank_{model_name}'])
    except (ValueError, TypeError):  #Catching more types of errors
        return np.nan  # Or another default value as needed


    zscore = row[f'zscore_{model_name}']
    if rank <= 8:
        conviction = max(-3, min(3, np.round(zscore * 4)/2 ))
    elif rank <=12: #Between 9 and 12
        conviction=0
    elif rank >= 13 :# >= 13

        conviction = max(-3, min(3, round_2x_to_half(zscore)))
    else:
        return np.nan
    return conviction 




# Main loop:
# Ensure 'Date' in model_list is datetime64[ns] type:
model_list['Date'] = model_list['Date'].dt.strftime('%Y-%m-%d')

for date_str, df in date_metric_dictionary.items():
    #Ensure also that your dictionnary keys are in datetime format, without time 
    date = pd.to_datetime(date_str).strftime('%Y-%m-%d')#Remove time from timestamp
    
    model_row = model_list[model_list['Date'] == date]

    if not model_row.empty:

        model_name = model_row['model'].iloc[0]
        date_metric_dictionary[date_str] = calculate_conviction_and_zscore(df.copy(), model_name) 

    else:
        print(f"No model found for {date}.")


No model found for 2021-03-18.
No model found for 2021-04-01.
No model found for 2021-05-03.
No model found for 2021-06-01.
No model found for 2021-07-01.
No model found for 2021-08-02.
No model found for 2021-09-01.
No model found for 2021-10-01.
No model found for 2021-11-01.
No model found for 2021-12-01.
No model found for 2022-01-03.
No model found for 2022-02-01.
No model found for 2022-03-01.
No model found for 2022-04-01.
No model found for 2022-05-02.
No model found for 2022-06-01.
No model found for 2022-07-01.
No model found for 2022-08-01.
No model found for 2022-09-01.
No model found for 2022-10-03.
No model found for 2022-11-01.
No model found for 2022-12-01.
No model found for 2023-01-03.
No model found for 2023-02-01.
No model found for 2023-03-01.
No model found for 2023-04-03.
No model found for 2023-05-01.
No model found for 2023-06-01.
No model found for 2023-07-03.
No model found for 2023-08-01.
No model found for 2023-09-01.
No model found for 2023-10-02.
No model

In [72]:

from xbbg import blp 
import pandas as pd
from dateutil.relativedelta import relativedelta

# Define the list of Bloomberg tickers
tickers = ["BRL BGN Curncy", "CLP BGN Curncy", "CNY CMPN Curncy", "COP REGN Curncy",
           "CZK BGN Curncy", "HUF BGN Curncy", "IDR CMPN Curncy", "MYR CMPN Curncy",
           "MXN BGN Curncy", "PEN BGN Curncy", "PHP CMPN Curncy", "PLN BGN Curncy",
           "RON BGN Curncy", "RUB BGN Curncy", "ZAR BGN Curncy", "THB BGN Curncy",
           "TRY BGN Curncy", "INR CMPN Curncy", "SGD BGN Curncy", "TWD BGN Curncy",
           "KRW CMPN Curncy"]

# Extract currency codes from tickers
currencies = [ticker[:3] for ticker in tickers]

# Get historical data for the past 5 years
end_date = pd.Timestamp.today()
start_date = end_date - relativedelta(years=5)

data_prices = blp.bdh(tickers, 'PX_LAST', start_date, end_date)
data_prices.columns = currencies
data_prices

returns = data_prices.shift()/data_prices -1 
returns = returns.dropna()
returns = returns.drop('RUB', axis=1).copy()

In [ ]:

# Define models and constants
models = ["RN", "CN", "SN", "RR", "CR", "SR"]
concentration_limit = 0.05
carry_cap = -0.003
lower_te_bound = 0.008
upper_te_bound = 0.012
# These values were missing, I have added them based on the 'correct' code.
i2_value = -0.003
k2_value = 0.005
ab38_value = 6
default = 0.03
ratio = [round(default /k2_value)]

# Calculate the average 3M Implied Vol once outside the loop
avg_implied_vol = df['3M Implied Vol'].mean()


# Loop through each model and calibrate position sizes
for model_name in models:
    for method in ["Agn", "Awa"]:
        model = f"{model_name}-{method}"
        conviction_col = f"conviction_{model_name}_{method}"
        standard_size = 0.03 if method == "Agn" else 0.035

        # Step 1: Initial size calculation (Corrected to match the first code)
        df[f'size_0_{model}'] = 100*df.apply(
            lambda row: (
                i2_value / row['Nominal_carry']
                if (
                    max(
                        min(
                            avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * ab38_value * k2_value / 2,
                            concentration_limit
                        ),
                        -concentration_limit
                    ) * row['Nominal_carry'] < i2_value
                )
                else (
                    max(
                        min(
                            avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * ab38_value * k2_value / 2,
                            concentration_limit
                        ),
                        -concentration_limit
                    )
                )
            ),
            axis=1
        )



        # Step 1 (cont.): Carry control (unchanged)
        df[f'size_1_{model}'] = df.apply(
            lambda row: row[f'size_0_{model}']
            if row[f'size_0_{model}'] * row['Nominal_carry'] >= carry_cap
            else carry_cap / row['Nominal_carry'],
            axis=1
        )

In [ ]:
### CHECK WHY MULTIPLY BY 100 TO OBTAIN THE RIGHT RESULT 
for model_name in models:
    for method in ["Agn", "Awa"]:
        model = f"{model_name}-{method}"
        conviction_col = f"conviction_{model_name}_{method}"
        standard_size = 0.03 if method == "Agn" else 0.035 
        implied_vol_sum = df['3M Implied Vol'].sum()

        for u in range(2, 7):  # u in [2, 3, 4, 5, 6]  (Steps 2-6)
            standard_size = 0.03 if method == "Agn" else 0.035  
            size_prev_col = f'size_{u-1}_{model}'
            size_curr_col = f'size_{u}_{model}'
            weights = df[size_prev_col]

            portfolio_returns = np.dot(returns.values, weights)
            annualized_std = np.std(portfolio_returns) * np.sqrt(252)
            df[f'std_{model}_step{u - 1}'] = annualized_std

            ex_ante_te = annualized_std

            if ex_ante_te < lower_te_bound and ex_ante_te != 0:
                standard_size *= math.ceil(lower_te_bound / ex_ante_te) #Update standard size iteratively
            elif ex_ante_te > upper_te_bound:
                standard_size *= math.floor(upper_te_bound / ex_ante_te)
            elif ex_ante_te == 0:  # Handle the zero case if needed.
                standard_size *= 1 # No change to standard size in this case


            df[size_curr_col] = 100*df.apply(
                lambda row: (
                    carry_cap / row['Nominal_carry']
                    if (
                        max(
                            min(
                                avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * standard_size * k2_value / 2,
                                concentration_limit
                            ),
                            -concentration_limit
                        ) * row['Nominal_carry'] < carry_cap
                    )
                    else (
                        max(
                            min(
                                avg_implied_vol / row['3M Implied Vol'] * row[conviction_col] * standard_size * k2_value / 2,
                                concentration_limit
                            ),
                            -concentration_limit
                        )
                    )
                ),
                axis=1
            )

            df[size_curr_col] = df.apply(
                lambda row: (
                    row[size_curr_col]
                    if row[size_curr_col] * row['Nominal_carry'] >= carry_cap
                    else carry_cap / row['Nominal_carry']
                ),
                axis=1
            )
            df[size_curr_col]  = df[size_curr_col].clip(lower=-concentration_limit, upper=concentration_limit)


    ### TAKE THE RESULT AND BOUND BETWEEN -5% and 5% 

